In [12]:
!apt apt install ffmpeg

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)


In [2]:
import pandas as pd
from multiprocessing import Pool, Value
from urllib.request import urlopen

import torch
import io
from sklearn.neighbors import NearestNeighbors
import numpy as np

import video_clip

In [2]:
NUM_PROCESS = 8
CLIPS_SIZE = 10_000

In [3]:
clips = pd.read_csv("../src/yappy_hackaton_2024_400k.csv")[:CLIPS_SIZE]

In [4]:
clips["id"] = clips["link"].apply(lambda x: hash(x) % 4242424242424237)

In [5]:
clips.head()

,link,description,id
0,https://cdn-st.rutubelist.ru/media/b0/e9/ef285...,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m...",971968710816746
1,https://cdn-st.rutubelist.ru/media/39/6c/b31bc...,🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #rob...,2064496738576893
2,https://cdn-st.rutubelist.ru/media/e9/e0/b47a9...,"#boobs , #красивыедевушки , #ass",3974149899574751
3,https://cdn-st.rutubelist.ru/media/87/43/b11df...,NaN,3360893729581392
4,https://cdn-st.rutubelist.ru/media/d1/e7/642dc...,NaN,1710849424916277


In [6]:
len(clips)

10000

In [7]:
assert clips["id"].unique().shape[0] == clips["id"].shape[0]

In [8]:
n = len(clips) // NUM_PROCESS
chunks = [clips[i:i+n] for i in range(0,len(clips) + 1,n)]

assert sum([len(x) for x in chunks]) == len(clips)


In [13]:
def video2embedding(video: io.BytesIO, model, vls_processor, device, reduction="mean"): 
    embedding = video_clip.get_all_video_embeddings([video], model, vls_processor, device)
    if reduction == "mean":
        embedding = embedding[0].mean(dim=1).ravel()
    return embedding

In [14]:
counter = Value('i', 0)

In [15]:
def f(process_idx: int, verbose: int = 20):
    eval_config = 'eval_configs/video_clip_v0.2.yaml'
    model, vis_processor = video_clip.load_model(eval_config)
    model = model.to("cuda:6")
    model = model.eval()

    with torch.no_grad():
        res = {}
        for idx, (_, line) in enumerate(chunks[process_idx].iterrows()):
            link = line["link"]
            name = line["id"]

            try:
                rsp = urlopen(link)
            except Exception as e:
                print("FUCK", e)
                continue

            embedding = video2embedding(io.BytesIO(rsp.read()), model, vis_processor, "cuda:6")

            res[name] = embedding.cpu().numpy()

            if idx != 0 and idx % verbose == 0:
                with counter.get_lock():
                    counter.value += verbose

                if process_idx == 0:
                    print(counter.value, end="\r")

    return res

In [16]:
with Pool(NUM_PROCESS) as p:
    result = p.map(f, list(range(NUM_PROCESS)))

Loading VIT
Loading VIT
Loading VIT
Loading VITLoading VIT

Loading VIT
Loading VIT
Loading VIT
Loading VIT Done
Loading Q-Former
Loading VIT DoneLoading VIT Done
Loading Q-Former

Loading Q-FormerLoading VIT Done

Loading Q-Former
Loading VIT Done
Loading Q-Former
Loading VIT Done
Loading Q-Former
Loading VIT Done
Loading Q-Former
Loading VIT Done
Loading Q-Former
Load first Checkpoint: ./models/askvideos_clip_v0.2.pth
Load first Checkpoint: ./models/askvideos_clip_v0.2.pth
Load first Checkpoint: ./models/askvideos_clip_v0.2.pth
Load first Checkpoint: ./models/askvideos_clip_v0.2.pth
Load first Checkpoint: ./models/askvideos_clip_v0.2.pth
Load first Checkpoint: ./models/askvideos_clip_v0.2.pth
Load first Checkpoint: ./models/askvideos_clip_v0.2.pth
Load first Checkpoint: ./models/askvideos_clip_v0.2.pth
FUCK <urlopen error [Errno 101] Network is unreachable>
FUCK <urlopen error [Errno 101] Network is unreachable>
FUCK <urlopen error [Errno -5] No address associated with hostname>
FUCK

In [17]:
id_to_url = {id: url for id, url in zip(clips["id"], clips["link"])}

In [18]:
joined_result = {}

for r in result:
    for key, embed in r.items():
        joined_result[id_to_url[key]] = embed

In [15]:
# import pickle


# with open("url_to_embed.pkl", "wb") as f:
#     pickle.dump(joined_result, f)

In [3]:
import pickle


with open("joined_embeddings.pkl", "rb") as f:
    joined_result = pickle.load(f)

In [14]:
joined_result[next(iter(joined_result.keys()))].shape

(1024,)

In [4]:
DEVICE = "cpu"# "cuda:6"

In [5]:
eval_config = 'eval_configs/video_clip_v0.2.yaml'
model, vis_processor = video_clip.load_model(eval_config, DEVICE)
model = model.to(DEVICE)
model = model.eval()


Loading VIT
Loading VIT Done
Loading Q-Former


/home/sivtsovdt/miniconda3/envs/hack-back/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Load first Checkpoint: ./models/askvideos_clip_v0.2.pth


In [6]:
text_embedding = video_clip.embed_text_itc(model, 'minecraft', DEVICE).cpu().numpy()

In [7]:
urls = []
embeddings = []
for u, e in joined_result.items():
    urls.append(u)
    embeddings.append(e)

neigh = NearestNeighbors(n_neighbors=10, metric="cosine")
neigh.fit(embeddings)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [8]:
indexes = neigh.kneighbors(text_embedding, 10, return_distance=False).ravel().tolist()
urls[indexes[0]]

'https://cdn-st.rutubelist.ru/media/b7/23/52d9d20f47ad9f2c78c1d1607674/fhd.mp4'

In [9]:
indexes = neigh.kneighbors(text_embedding, 10, return_distance=False).ravel().tolist()

In [10]:
indexes

[148833, 49737, 254844, 222368, 216988, 302189, 140949, 16711, 10842, 315238]

In [11]:
urls[indexes[0]]

'https://cdn-st.rutubelist.ru/media/b7/23/52d9d20f47ad9f2c78c1d1607674/fhd.mp4'

In [12]:
urls[indexes[5]]

'https://cdn-st.rutubelist.ru/media/19/38/cf87da934c5aa7cdd8e34fa3655e/fhd.mp4'